In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import trees as q2
from multiprocessing import Pool
from itertools import repeat
import scipy.stats

In [2]:
def Calcu_Accu(label,pred,threshold):
    count=0
    for i in range(len(label)):
        if pred[i]<threshold and label[i]==0:
            count+=1
        elif pred[i]>threshold and label[i]==1:
            count+=1
    return count/float(len(label))

In [3]:
# def tanh(x):
#     return np.tanh(x)
def sigmoid(x):
    return 1/(1+np.exp(-x))
# def tanh_deriv(x):
#     return 1.0 - np.tanh(x)*np.tanh(x)
def sigmoid_deriv(x):
    return sigmoid(x)*(1-sigmoid(x))
# class NN(object):
#     def __init__(self, layers, activation):
#         """
#         :param layers: list specify the number of neurons in each layer
#         :param activation: only supports sigmoid and tanh
#         """
#         self.layers=layers
#         if activation == 'sigmoid':
#             self.activation = sigmoid
#             self.activation_deriv = sigmoid_deriv
#             self.threshold=0.5
#         elif activation == 'tanh':
#             self.activation = tanh
#             self.activation_deriv = tanh_deriv
#             self.threshold = 0
#         self.weights = []
#         for i in range(1, len(layers)-1):
#             self.weights.append(np.random.random((layers[i - 1] + 1, layers[i] + 1))-0.5)
#         self.weights.append(np.random.random((layers[len(layers) - 2] + 1, layers[len(layers)-1])) - 0.5)
#     def fit(self, X, y, learning_rate, epochs):
#         X = np.atleast_2d(X)#Number of examples*features
#         temp = np.ones([X.shape[0], X.shape[1] + 1])#add 1 vector for bias
#         temp[:, 0:-1] = X
#         X = temp
#         y = np.array(y)
#         update_weight=[0]*len(self.weights)
#         for k in range(epochs):
#             #combine = list(zip(X,y))
#             #random.shuffle(combine)
#             #X[:], y[:] = zip(*combine)
#             for i in range(len(X)):
#                 example=X[i]
#                 pred=[example]

#                 for l in range(len(self.weights)):
#                     #if l==0 and i==0:
#                     #    print(self.weights[l])
#                     #print(pred[l].shape)
#                     pred.append(self.activation(np.dot(pred[l],self.weights[l])))

#                 error=y[i]-pred[-1]
#                 deltas=[error*self.activation_deriv(pred[-1])]
#                 for l in range(len(pred) - 2, 0, -1):
#                     deltas.append(deltas[-1].dot(self.weights[l].T) * self.activation_deriv(pred[l]))
#                 deltas.reverse()
#                 for l in range(len(self.weights)):
#                     layer = np.atleast_2d(pred[l])
#                     delta = np.atleast_2d(deltas[l])
#                     update_weight[l] += learning_rate * layer.T.dot(delta)
#             for l in range(len(self.weights)):
#                 self.weights[l]+=update_weight[l]

#             #Calculate the accuracy
#             #pred = self.pred_array(X[:,:-1])
#             #train_accu = Calcu_Accu(y, pred, self.threshold)
#             #print('%d epoch: Training Accuracy NN: %.2f' % (k,train_accu))
#     def predict(self, x):
#         x = np.array(x)
#         temp = np.ones(x.shape[0] + 1)
#         temp[0:-1] = x
#         a = temp
#         for l in range(0, len(self.weights)):
#             a = self.activation(np.dot(a, self.weights[l]))
#         return a
#     def pred_array(self,array):
#         pred=[]
#         for i in range(len(array)):
#             result=self.predict(array[i])
#             pred.append(result)
#         return pred
def predict(x, weights):
    x = np.array(x)
    temp = np.ones(x.shape[0] + 1)
    temp[0:-1] = x
    a = temp
    for l in range(0, len(weights)):
        a = sigmoid(np.dot(a, weights[l]))
    return a

def pred_array(array, weights):
    pred=[]
    for i in range(len(array)):
        result=predict(array[i], weights)
        pred.append(result)
    return pred

def fit(X, y, weights, learning_rate, epochs):
    X = np.atleast_2d(X)#Number of examples*features
    temp = np.ones([X.shape[0], X.shape[1] + 1])#add 1 vector for bias
    temp[:, 0:-1] = X
    X = temp
    y = np.array(y)
    update_weight=[0]*len(weights)
    for k in range(epochs):
        for i in range(len(X)):
            example=X[i]
            pred=[example]

            for l in range(len(weights)):
                pred.append(sigmoid(np.dot(pred[l],weights[l])))

            error=y[i]-pred[-1]
            deltas=[error*sigmoid_deriv(pred[-1])]
            for l in range(len(pred) - 2, 0, -1):
                deltas.append(deltas[-1].dot(weights[l].T) * sigmoid_deriv(pred[l]))
            deltas.reverse()
            for l in range(len(weights)):
                layer = np.atleast_2d(pred[l])
                delta = np.atleast_2d(deltas[l])
                update_weight[l] += learning_rate * layer.T.dot(delta)
        for l in range(len(weights)):
            weights[l]+=update_weight[l]
    return weights

def neural_net(layers, train_set):
    weights = []
    for i in range(1, len(layers)-1):
        weights.append(np.random.random((layers[i - 1] + 1, layers[i] + 1))-0.5)
    weights.append(np.random.random((layers[len(layers) - 2] + 1, layers[len(layers)-1])) - 0.5)

    weights = fit(np.array(train_set.iloc[:,:-1]), np.array(train_set.iloc[:,-1]), weights, 0.001,50)
    return weights

def get_model_accuracy(train_set, test_set, weights, threshold):
    train_pred = pred_array(np.array(train_set.iloc[:,:-1]), weights)
    train_accu=Calcu_Accu(np.array(train_set.iloc[:,-1]),train_pred,threshold)
    print('Training Accuracy NN: %.2f' % train_accu)

    test_pred = pred_array(np.array(test_set.iloc[:,:-1]), weights)
    test_accu=Calcu_Accu(np.array(test_set.iloc[:,-1]),test_pred,threshold)
    print('Testing Accuracy NN: %.2f' % test_accu)


In [4]:
train_reader = pd.read_csv('trainingSet.csv')
test_reader = pd.read_csv('testSet.csv')
activation = 'sigmoid'
# func = 'sigmoid'
threshold = 0.5
features=len(train_reader.columns)-1
# Network=NN([features,50,20,1],func)
weights = neural_net([features,50,20,1], train_reader)
# Network.fit(np.array(train_reader.iloc[:,:-1]),np.array(train_reader.iloc[:,-1]),0.001,50)
# pred=Network.pred_array(np.array(train_reader.iloc[:,:-1]))

# train_accu=Calcu_Accu(np.array(train_reader.iloc[:,-1]),pred,threshold)
# pred=Network.pred_array(np.array(test_reader.iloc[:,:-1]))
# test_accu=Calcu_Accu(np.array(test_reader.iloc[:,-1]),pred,threshold)
# print('Training Accuracy NN: %.2f' % train_accu)
# print('Testing Accuracy NN: %.2f' % test_accu)
get_model_accuracy(train_reader, test_reader, weights, threshold)

Training Accuracy NN: 0.74
Testing Accuracy NN: 0.72


In [5]:
!python preprocess-assg4.py

In [6]:
!python neuralNetwork.py

Training Accuracy NN: 0.72
Testing Accuracy NN: 0.71


In [7]:
!python cv_frac.py

Current fraction: 0.05
Current fold: 0
DT acc: 0.6173076923076923
In BT
Bagging acc 0.6692307692307692
In RF
Random Forest acc 0.6480769230769231
Current fold: 1
DT acc: 0.6903846153846154
In BT
Bagging acc 0.7403846153846154
In RF
Random Forest acc 0.7326923076923076
Current fold: 2
DT acc: 0.7038461538461539
In BT
Bagging acc 0.7096153846153846
In RF
Random Forest acc 0.6846153846153846
Current fold: 3
DT acc: 0.7192307692307692
In BT
Bagging acc 0.7173076923076923
In RF
Random Forest acc 0.7211538461538461
Current fold: 4
DT acc: 0.7269230769230769
In BT
Bagging acc 0.7230769230769231
In RF
Random Forest acc 0.7384615384615385
Current fold: 5
DT acc: 0.7019230769230769
In BT
Bagging acc 0.7211538461538461
In RF
Random Forest acc 0.7269230769230769
Current fold: 6
DT acc: 0.6576923076923077
In BT
Bagging acc 0.698076923076923
In RF
Random Forest acc 0.7153846153846154
Current fold: 7
DT acc: 0.7269230769230769
In BT
Bagging acc 0.7038461538461539
In RF
Random Forest acc 0.71923076923

In [8]:
!python trees.py trainingSet.csv testSet.csv 1

Training Accuracy DT: 0.77
Testing Accuracy DT: 0.72


In [9]:
!python trees.py trainingSet.csv testSet.csv 2

In BT
Training Accuracy BT: 0.79
Testing Accuracy BT: 0.75


In [10]:
!python trees.py trainingSet.csv testSet.csv 3

In RF
Training Accuracy RF: 0.76
Testing Accuracy RF: 0.73


In [12]:
!python cv_depth.py

Current depth: 3
Current fold: 0
Current fold: 1
Current fold: 2
Current fold: 3
Current fold: 4
Current fold: 5
Current fold: 6
Current fold: 7
Current fold: 8
Current fold: 9
Current depth: 5
Current fold: 0
Current fold: 1
Current fold: 2
Current fold: 3
Current fold: 4
Current fold: 5
Current fold: 6
Current fold: 7
Current fold: 8
Current fold: 9
Current depth: 7
Current fold: 0
Current fold: 1
Current fold: 2
Current fold: 3
Current fold: 4
Current fold: 5
Current fold: 6
Current fold: 7
Current fold: 8
Current fold: 9
Current depth: 9
Current fold: 0
Current fold: 1
Current fold: 2
Current fold: 3
Current fold: 4
Current fold: 5
Current fold: 6
Current fold: 7
Current fold: 8
Current fold: 9
Figure(640x480)
Value of p: 0.06426597760804982
Accepting null hypothesis
Current depth: 3
Current fold: 0
Current fold: 1
Current fold: 2
Current fold: 3
Current fold: 4
Current fold: 5
Current fold: 6
Current fold: 7
Current fold: 8
Current fold: 9
Current depth: 5
Current fold: 0
Current 

In [13]:
!python cv_numtrees.py

^C
